In [77]:
import numpy as np


In [78]:
class TicTacToe:
  def __init__(self) -> None:
    self.column_size=3
    self.row_size=3
    self.action_size=self.column_size*self.row_size
  def get_initial_state(self):
    return np.zeros((self.column_size,self.row_size))
  def get_next_state(self,state,action,player):
    row=action//self.column_size;
    column=action % self.column_size;
    state[row,column]=player;
    return state;
  def get_valid_state(self,state):
    return (state==0).reshape(-1).astype(np.uint8)
  def check_win(self,state,action):
    row=action//self.column_size;
    col=action%self.column_size
    diagnol_sum=np.diag(state).sum()
    anti_diagnol_sum=np.diag(np.fliplr(state)).sum()
    row_sum=state[row,:].sum()
    col_sum=state[:,col].sum()
    player=state[row,col]
    print(self.column_size,player)
    if diagnol_sum==player*self.column_size or anti_diagnol_sum==player*self.column_size:
      return True
    if row_sum==player*self.column_size or col_sum==player*self.column_size:
      return True
    return False
  def get_value_terminated(self,state,action):
    result=self.check_win(state,action)
    if result ==True:
      return 1, True
    if np.sum(self.get_valid_state(state))==0:
      return 0,True
    return 0,False

In [79]:
 state=np.zeros((4,4))
 (state==0).reshape(-1).astype(np.uint8)

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=uint8)

In [80]:
game=TicTacToe()
state=game.get_initial_state()
player=1;
for i in range(10):
  actions=game.get_valid_state(state)
  action=np.random.choice(np.where(actions==1)[0])
  state=game.get_next_state(state,action,player)
  reward,termination=game.get_value_terminated(state,action)
  if termination==True:
    print(reward)
    break;
  player=-player;


3 1.0
3 -1.0
3 1.0
3 -1.0
3 1.0
3 -1.0
3 1.0
3 -1.0
3 1.0
1


In [81]:
game=TicTacToe()
state=game.get_initial_state()
actions=game.get_valid_state(state)
action=np.random.choice(np.where(actions==1)[0])
game.check_win(state,action)

3 0.0


True

In [82]:
game1=TicTacToe()
state=game1.get_initial_state()
actions=game1.get_valid_state(state)
np.random.choice(np.where(actions==1)[0])
# np.random(np.choice(np.where(actions==1))

np.int64(8)

In [83]:


constant=2/3

In [84]:
class Node:
  def __init__(self,player,state,prior_policy) -> None:
    self.player=int(player);
    # Store state as a NumPy array
    self.state=state;
    self.prior_policy=prior_policy;
    self.policy=[]
    self.visit_count=0;
    self.total_reward=0;
    self.children=[]

  # rollout phase
  def simulate(self,game):
    state=self.state.copy() # Create a copy to avoid modifying the original state
    player=self.player;
    for i in range(10):
     actions=game.get_valid_state(state)
     # Check if there are any valid actions before choosing
     if np.sum(actions) == 0:
        # If no valid actions and the game is not terminated, it's a draw
        reward, termination = game.get_value_terminated(state, -1) # Pass a dummy action
        self.total_reward += reward
        return reward
     action=np.random.choice(np.where(actions==1)[0])
     state=game.get_next_state(state,action,player)
     reward,termination=game.get_value_terminated(state,action)
     if termination==True:
       self.total_reward+=reward
       return reward
     player=-player;
    # If the loop finishes without termination, it's a draw
    return 0


  # now check if the node is the leaf node
  def is_leaf(self):
    return len(self.children)==0
  # now expand
  def expand(self,game):
    actions=game.get_valid_state(self.state)
    if self.is_leaf()!=True:
      return
    for i in range(len(actions)):
      if actions[i]==1:
        # Pass the next state as a NumPy array
        self.children.append(Node(-self.player,game.get_next_state(self.state.copy(),i,self.player),1/np.sum(actions))) # Use copy to avoid modifying the parent state
  def calculate_ucb(self):
    if self.visit_count==0:
      return 1e10
    reward_mean=self.total_reward/self.visit_count
    ucb=reward_mean+constant*np.sqrt(np.log(self.visit_count)/self.visit_count)
    return ucb

In [85]:
def selection(node):
  #terminating condition
  if(node.is_leaf()):
     reward=node.simulate(game)
     node.total_reward+=reward
     node.visit_count+=1;
     node.expand(game)
     return reward
  #choose the branch with the maximum ucb that is upper confidence bound
  max_ucb_child=None
  max_ucb=-1e10
  for child in node.children:
    ucb=child.calculate_ucb()
    if(ucb>=max_ucb):
      max_ucb_child=child
      max_ucb=ucb
  reward=selection(max_ucb_child)
  node.total_reward+=reward
  node.visit_count+=1
  return reward

In [86]:
#Initialize the Root and the game
game=TicTacToe()
state=game.get_initial_state()
root=Node(1,state,1)
number_of_simulations=3
current=root

In [87]:
for epoch in range(number_of_simulations):
  selection(root)

3 1.0
3 -1.0
3 1.0
3 -1.0
3 1.0
3 -1.0
3 1.0
3 -1.0
3 1.0
3 -1.0
3 1.0
3 -1.0
3 1.0
3 -1.0
3 1.0
3 -1.0
3 1.0
3 -1.0
3 1.0
3 -1.0
3 1.0


# Task
Visualize the MCTS tree using a suitable tool.

## Install necessary libraries

### Subtask:
Install libraries for tree visualization (e.g., `graphviz`).


**Reasoning**:
The subtask requires installing the `graphviz` library, which can be done using pip in a code cell.



In [88]:
%pip install graphviz

## Implement tree traversal

### Subtask:
Write a function to traverse the MCTS tree and collect node information.


**Reasoning**:
Define a function to traverse the MCTS tree and collect node information, including state, visit count, total reward, and parent-child relationships.



In [89]:
def traverse_tree(node, node_list, parent_id=None):
    """Traverses the MCTS tree and collects node information."""
    node_id = len(node_list)
    node_info = {
        'id': node_id,
        'parent_id': parent_id,
        'state': node.state.tolist(), # Convert numpy array to list for serialization
        'visit_count': node.visit_count,
        'total_reward': node.total_reward,
        'prior_policy': node.prior_policy,
        'player': node.player
    }
    node_list.append(node_info)

    for child in node.children:
        traverse_tree(child, node_list, node_id)

node_list = []
traverse_tree(root, node_list)


## Generate graph data

### Subtask:
Format the collected node information into a graph data structure.


**Reasoning**:
Import the graphviz library and create a directed graph object.



In [90]:
import graphviz
mcts_graph = graphviz.Digraph()

**Reasoning**:
Iterate through the collected node information and add nodes and edges to the graph.



In [91]:
for node_info in node_list:
    state_str = str(node_info['state']).replace('[[', '[').replace(']]', ']').replace(' ', '')
    prior_policy_str = f"{node_info['prior_policy']:.2f}" if node_info['prior_policy'] is not None else "N/A"
    node_label = f"ID: {node_info['id']}\nState: {node_info['state']}\nVisits: {node_info['visit_count']}\nReward: {node_info['total_reward']:.2f}\nPrior: {prior_policy_str}\nPlayer: {node_info['player']}"
    mcts_graph.node(str(node_info['id']), label=node_label)
    if node_info['parent_id'] is not None:
        mcts_graph.edge(str(node_info['parent_id']), str(node_info['id']))

In [92]:
mcts_graph.render('mcts_tree_shortened', view=True, format='png')

'mcts_tree_shortened.png'

## Visualize the tree

### Subtask:
Use the installed library to generate a visualization of the tree from the graph data.


**Reasoning**:
Render the generated graph data to a file.



In [93]:
mcts_graph.render('mcts_tree', view=True, format='png')

'mcts_tree.png'

## Summary:

### Data Analysis Key Findings

*   The `graphviz` library was already installed in the environment.
*   A Python function successfully traversed the MCTS tree and collected node information, including node ID, parent ID, state, visit count, total reward, prior policy, and player.
*   The collected node information was successfully formatted into a directed graph structure using `graphviz`.
*   The MCTS tree visualization was generated as a PNG file named 'mcts\_tree.png'.

### Insights or Next Steps

*   Review the generated `mcts_tree.png` file to visually inspect the structure and key metrics of the MCTS tree.
*   Consider adding more detailed information or conditional styling to the node labels or edges in the graph visualization to highlight specific aspects of the MCTS process (e.g., winning nodes, high visit counts, specific states).
